In [1]:
import pyrealsense2 as rs
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### 1. 데이터 정합하는 함수

In [5]:
def transform_and_save_point_cloud(file_path, degree, radius, output_path=None):
    """
    주어진 각도와 반지름에 따라 Point Cloud를 회전+이동 변환하여 저장하는 함수.

    Parameters:
    - file_path: 입력 포인트 클라우드 파일 경로 (.pcd)
    - degree: 회전 각도 (도 단위)
    - radius: 카메라 원 반지름 (m)
    - output_path: 저장 경로 (기본값은 'cloud_{degree}_aligned.pcd')
    """
    # 포인트 클라우드 로드
    pcd = o3d.io.read_point_cloud(file_path)

    # 각도 → 라디안
    theta = np.radians(degree)
    
    # 회전 행렬 (yaw 회전)
    R = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    
    # 카메라 위치 계산 (반지름 적용)
    tx = radius * np.sin(theta)
    tz = radius * np.cos(theta)

    # 4x4 변환 행렬 구성
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = [-tx, 0.0, -tz] # 이동 벡터

    # 변환 적용
    pcd.transform(T)

    # 저장 경로 설정
    if output_path is None:
        output_path = f"cloud_{degree}_aligned_test1.pcd"

    o3d.io.write_point_cloud(output_path, pcd)
    print(f"✓ 저장 완료: {output_path}")


##### 배경 제거 파일 불러와 정합하고 시각화하는 코드 

In [6]:
import os

# 정합할 각도와 반지름
degrees = [0, 90, 180, 270]
radius = 0.5  # 반지름 (m)

# 기본 경로 설정
base_dir = r"D:\repos\my_librealsense\pcd_data\bgm"

# 각 PCD 파일에 대해 정합 수행
for deg in degrees:
    file_path = os.path.join(base_dir, f"cloud_{deg}_test1_shoes_result.pcd")
    output_path = os.path.join(base_dir, f"cloud_{deg}_aligned_test1.pcd")
    transform_and_save_point_cloud(file_path, deg, radius, output_path)


✓ 저장 완료: D:\repos\my_librealsense\pcd_data\bgm\cloud_0_aligned_test1.pcd
✓ 저장 완료: D:\repos\my_librealsense\pcd_data\bgm\cloud_90_aligned_test1.pcd
✓ 저장 완료: D:\repos\my_librealsense\pcd_data\bgm\cloud_180_aligned_test1.pcd
✓ 저장 완료: D:\repos\my_librealsense\pcd_data\bgm\cloud_270_aligned_test1.pcd


In [7]:
import open3d as o3d
import os

# 정합된 PCD 파일 경로 정보
degrees = [0, 90, 180, 270]
base_dir = r"D:\repos\my_librealsense\pcd_data\bgm"

# 병합 시작
combined_pcd = o3d.io.read_point_cloud(os.path.join(base_dir, f"cloud_{degrees[0]}_aligned_test1.pcd"))
for degree in degrees[1:]:
    aligned_path = os.path.join(base_dir, f"cloud_{degree}_aligned_test1.pcd")
    aligned = o3d.io.read_point_cloud(aligned_path)
    combined_pcd += aligned

# 좌표축 추가
coord = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.1)

# 시각화
o3d.visualization.draw_geometries([combined_pcd, coord])


[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.
